In [ ]:
# coding: utf8

#Import

In [ ]:
import os
import sys
import json
import pprint
from pyes import *
import pandas as pd
pp = pprint.PrettyPrinter(indent=4)
from datetime import date, datetime

#Confinguration

In [ ]:
HERE = os.getcwd() + '/'
PROJECTS_CSV = u'demos.csv' 

#Class project

In [ ]:
class Demo(object):
    """
    """
    
    def __init__(self, title='title of the demo', desc='...', Type='bonnus', date=date.today(), html=u'index.html'):
        self._id = ''
        
        self.title = title
        self.date = date
        self.source = html
        self.demoType = Type
        self.description = desc
        
        self.tools = []
        self.credits = []
        self.seeAlso = []
        self.score = 1.0
    
    
    @classmethod
    def fetch(self, idx, elastic=ES('127.0.0.1:9200')):
        doc = elastic.get('lmd', 'demo', idx)
        
        d = Demo(doc.title, doc.description, doc.Type, datetime.strptime(doc.date,"%Y-%m-%d"), doc.source)
        
        d.addTools(doc.tools)
        d.addCredits(doc.credits)
        #d.seeAlso(doc.seeAlso)
        
        d._id = idx
        
        return d
    
    def __addAttributes(self, attr, values):
        for v in values:
            if v not in attr: attr.append(v)
                
    def addTools(self, values):
        self.__addAttributes(self.tools, values)
        
    def addCredits(self, values):
        self.__addAttributes(self.credits, values)
    
    def addSeeAlso(self, values):
        self.__addAttributes(self.seeAlso, values)
                
    @staticmethod
    def linkAlias(field, listsep="|", attrsep=",", na=''):
        tab = field.split(listsep)
        if field == na or len(tab) < 1:
            return list()
        else:
            return map(lambda y: {u'alias': y.split(attrsep)[0],\
                                  u'link': y.split(attrsep)[1]}, tab)
        
        
    @property
    def show(self):
        return {u'title': self.title,
           u'description': self.description,
           u'type': self.demoType,
           u'date': self.date.strftime(u'%Y-%m-%d'),
           u'source': self.source,
           u'tools': self.tools,
           u'credits': self.credits,
           u'seeAlso': self.seeAlso
          }


#Loader

### connection

In [ ]:
esConn = ES('127.0.0.1:9200')

In [ ]:
#add mapping
mapping = {}

In [ ]:
df = pd.read_csv(HERE + PROJECTS_CSV, sep=';', lineterminator=u'\r')

In [66]:
df = df.fillna(value='')
df.head()

,id,title,description,type,date,source,tools,seeAlso,credits
0,_carcrashone,Paris car accidents 12/13,Which types of vehicule are involved in accide...,graph,20/04/15,cars_one.html,python|spark|D3.js,"map on same subject,/player/_carcrashtwo|#link...","thx to @kentenglish for all the chart tips,htt..."
1,_carcrashtwo,Paris car accidents per rings,Lets have a look the accident distribution in ...,graph,21/04/15,cars_two.html,python|spark|D3.js,"pie on the same subject,/player/_carcrashone|l...",
2,_composite,Design pattern move: Composite,,code,23/06/15,,,,
3,_blank2,a blank demo,X,ml,,,,,
4,_blank,a blank demo2,X,db,,,,,


In [ ]:
for idx, p in df.iterrows():
    d = Demo(title=p['title'], Type=p['type'], html=p['source'])
    d.description = p['description']
    d.addTools(p['tools'].split('|'))
    d.addCredits(Demo.linkAlias(p['credits']))
    d.addSeeAlso(Demo.linkAlias(p['seeAlso']))
    d.id = p['id']
    try:
        d.date = datetime.strptime(p[u'date'], u'%d/%m/%y')
    except:
        d.date = datetime.today()
        
    #pp.pprint(d.show)
    esConn.index(d.show, 'lmd', 'demo', p['id'])
    

esConn.index(d.show, 'lmd', 'demo')

#Update 

### find the document

In [ ]:
doc = Demo.fetch('AU8jgcj2gzIYf3juNf_8', esConn)

In [ ]:
d.show

### Perform update

In [ ]:
doc.tools = [u'Spark', u'D3.js']

### Save

In [ ]:
esConn.index(doc.show, u'lmd', u'demo', doc._id)